Aiming to have a package that generates simple SLiM scripts with all the functionality that msprime has + mutations.

### Example scenario:

A chromosome of length 10 000 000 bp.
Admixed population is a 70:30 mixture of 2 ancestral populations 50 generations ago, and 2\% (0.6\%:1.4\%) of the population in each generation are replaced by migrants from these unadmixed populations.

In [1]:
import slime
import msprime

Let's initialize the simulation. We will simulate chromosomes of 10 000 000 bp from a population that was produced by admixture 50 generations ago, and save the result into a file `examples/example-script.slim`.

In [2]:
my_script = slime.RecentHistory(final_gen = 50, chrom_length = 1e7, 
                                outfile = "examples/example-script.slim")

my_script.print_script()


initialize(){
    initializeSLiMModelType("WF");
    initializeTreeSeq();
    initializeMutationRate(0);
    initializeMutationType("m0", 0.500000, "f", 0.000000);
    initializeGenomicElementType("g1", c(m0), c(1.0));
    initializeGenomicElement(g1, 0, 10000000);
}    

1 early(){      
}

50 late(){
    sim.treeSeqOutput("examples/example-script.slim");
    sim.simulationFinished();
}


Next, let's add in the population configurations.
The ancestral populations must be added first.
We'll make `msprime.PopulationConfiguration` objects for these and add them to the script using the `add_reference_population()` method.

Note that the growth rates won't be added into the script until later.

In [3]:
config_references = msprime.PopulationConfiguration(0, 1000, growth_rate = 0.05)

my_script.add_reference_population(config_references, 'Population0')
my_script.add_reference_population(config_references, 'Population1')

my_script.print_script()


initialize(){
    initializeSLiMModelType("WF");
    initializeTreeSeq();
    initializeMutationRate(0);
    initializeMutationType("m0", 0.500000, "f", 0.000000);
    initializeGenomicElementType("g1", c(m0), c(1.0));
    initializeGenomicElement(g1, 0, 10000000);
}    

1 early(){      
    sim.addSubpop("p0", 1000);
    sim.addSubpop("p1", 1000);
}

50 late(){
    sim.treeSeqOutput("examples/example-script.slim");
    sim.simulationFinished();
}


Next, let's add in the admixed population. We'll need to input a `msprime.PopulationConfiguration` object as well as a list of admixture proportions.

In [4]:
config_admixed = msprime.PopulationConfiguration(0, 1000, growth_rate = 0.01)

my_script.add_admixed_population(config_admixed, 'PopulationAdm', [0.3, 0.7],
                                single_pulse = False, migration_rate = 0.02)

my_script.print_script()


initialize(){
    initializeSLiMModelType("WF");
    initializeTreeSeq();
    initializeMutationRate(0);
    initializeMutationType("m0", 0.500000, "f", 0.000000);
    initializeGenomicElementType("g1", c(m0), c(1.0));
    initializeGenomicElement(g1, 0, 10000000);
}    

1 early(){      
    sim.addSubpop("p0", 1000);
    sim.addSubpop("p1", 1000);
}

1 late(){
    sim.addSubpop("p2", 1000);
    p2.setMigrationRates(c(p0,p1),c(0.300000,0.700000));
}

2 late(){
    p2.setMigrationRates(c(p0,p1,p2),c(0.006000,0.014000,0.980000));
}

50 late(){
    sim.treeSeqOutput("examples/example-script.slim");
    sim.simulationFinished();
}


Finally, we'll add in other demographic events. (Still working on this bit).

For example, say we want to change the growth rate of the admixed population in the 20th and 40th generation. We'll make a `msprime.PopulationParametersChange` object and add this to the script using the `add_demographic_events()` method.

In [5]:
change1 = msprime.PopulationParametersChange(20, growth_rate =  .3, population_id = 2)
change2 = msprime.PopulationParametersChange(40, growth_rate =  .1, population_id = 2)

my_script.add_demographic_events([change1, change2])

my_script.print_script()


initialize(){
    initializeSLiMModelType("WF");
    initializeTreeSeq();
    initializeMutationRate(0);
    initializeMutationType("m0", 0.500000, "f", 0.000000);
    initializeGenomicElementType("g1", c(m0), c(1.0));
    initializeGenomicElement(g1, 0, 10000000);
}    

1 early(){      
    sim.addSubpop("p0", 1000);
    sim.addSubpop("p1", 1000);
}

1 late(){
    sim.addSubpop("p2", 1000);
    p2.setMigrationRates(c(p0,p1),c(0.300000,0.700000));
}

2 late(){
    p2.setMigrationRates(c(p0,p1,p2),c(0.006000,0.014000,0.980000));
}

20:39 {
    pop 0 has growth rate 0.050000;
    pop 1 has growth rate 0.050000;
    pop 2 has growth rate 0.300000;
}

40:49 {
    pop 0 has growth rate 0.050000;
    pop 1 has growth rate 0.050000;
    pop 2 has growth rate 0.100000;
}

50 late(){
    sim.treeSeqOutput("examples/example-script.slim");
    sim.simulationFinished();
}
